In [1]:
import requests
import pandas as pd
%load_ext kedro.ipython

[11/01/24 12:43:00] INFO     Resolved project path as: /home/pablo/dev/CIC/dw-cic/kedro/kedro-cic.  ]8;id=970223;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=808871;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py#146\146]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[11/01/24 12:43:01] INFO     Kedro project kedro-unlp                                               ]8;id=757416;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=365803;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py#115\115]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=234544;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=170569;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py#116\116]8;;\
                             'pipelines'                                                                           

[11/01/24 12:43:02] INFO     Registered line magic 'run_viz'                                        ]8;id=850828;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=457954;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py#122\122]8;;\

In [ ]:
institution_ror = catalog.load('params:openalex_fetch_options.institution_ror')
#env = catalog.load('params:fetch_options.env')
env = 'dev'

                    INFO     Loading data from params:openalex_fetch_options.institution_ror    ]8;id=162442;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.kedro-env/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=170061;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.kedro-env/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:fetch_options.env (MemoryDataset)...      ]8;id=125710;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.kedro-env/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=267681;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.kedro-env/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\

In [3]:
env

'dev'

In [ ]:
def fetch_work_openalex(institution_ror, env):
    cursor = '*'
    base_url = 'https://api.openalex.org/works?filter=institutions.ror:{}&cursor={}'
    iteration_limit = 5
    iteration_count = 0

    url = base_url.format(institution_ror, cursor)
    api_response = requests.get(url).json()

    print(f'Iteration count: {iteration_count}')
    print(f'GET {url}')

    # creo dataframe con las columnas del primer resultado 
    df = pd.DataFrame.from_dict(api_response['results'])

    # update cursor
    cursor = api_response['meta']['next_cursor']
    url = base_url.format(institution_ror, cursor)

    while cursor:

        if env == 'dev' and iteration_count >= iteration_limit:
            break

        # request api with updated cursor
        url = base_url.format(institution_ror, cursor)

        iteration_count += 1
        print(f'Iteration count: {iteration_count}')
        print(f'GET {url}')

        api_response = requests.get(url).json()

        df_tmp = pd.DataFrame.from_dict(api_response['results'])

        df = pd.concat([df, df_tmp])

        # update cursor
        cursor = api_response['meta']['next_cursor']

    return df, df.head(1000)

In [ ]:
df, df_dev = fetch_work_openalex(institution_ror, env)
df

Iteration count: 0
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=*
Iteration count: 1
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=Ils5OS4wLCAxOTYsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMTk5MjMyMTAyNSddIg==
Iteration count: 2
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=Ils5OS4wLCA5MiwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1cyMDQwNzEwNTgxJ10i
Iteration count: 3
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=Ils5OC4wLCA5NSwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1cyMTI4MjMxNjc2J10i
Iteration count: 4
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=Ils5OC4wLCAzNSwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1czMTM1ODI0NjMzJ10i
Iteration count: 5
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=Ils5Ny4wLCA3MiwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1cyMDE0ODQzMDg5J10i


,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,datasets,versions,referenced_works_count,referenced_works,related_works,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date
0,https://openalex.org/W2089852843,https://doi.org/10.1016/s0378-4371(98)00437-3,The role of constraints within generalized non...,The role of constraints within generalized non...,1998,1998-12-01,{'openalex': 'https://openalex.org/W2089852843...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,[],[],74,"[https://openalex.org/W1553236419, https://ope...","[https://openalex.org/W620348747, https://open...","{'The': [0], 'Gibbs–Jaynes': [1], 'path': [2],...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 42}, {'year'...",2024-10-29T17:53:52.050249,2016-06-24
1,https://openalex.org/W3037375387,https://doi.org/10.1016/j.envint.2020.105876,Short-term exposure to particulate matter (PM1...,Short-term exposure to particulate matter (PM1...,2020,2020-06-23,{'openalex': 'https://openalex.org/W3037375387...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",review,...,[],[],84,"[https://openalex.org/W1525116145, https://ope...","[https://openalex.org/W626601394, https://open...","{'Air': [0], 'pollution': [1, 87, 155], 'is': ...",https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2024, 'cited_by_count': 138}, {'year...",2024-10-31T17:56:04.919810,2020-07-02
2,https://openalex.org/W2032143374,https://doi.org/10.1016/s0023-6438(03)00088-4,Antimicrobial activity of essential oils on th...,Antimicrobial activity of essential oils on th...,2003,2003-11-01,{'openalex': 'https://openalex.org/W2032143374...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,[],[],37,"[https://openalex.org/W102433214, https://open...","[https://openalex.org/W4381740914, https://ope...","{'The': [0, 16, 80], 'antimicrobial': [1, 48, ...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 39}, {'year'...",2024-10-23T20:08:55.058764,2016-06-24
3,https://openalex.org/W2154507013,https://doi.org/10.1126/science.1134796,Formation of a Nematic Fluid at High Fields in...,Formation of a Nematic Fluid at High Fields in...,2006,2006-11-24,{'openalex': 'https://openalex.org/W2154507013...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,[],[https://openalex.org/W2154507013],34,"[https://openalex.org/W1523859489, https://ope...","[https://openalex.org/W54078636, https://opena...","{'In': [0], 'principle,': [1], 'a': [2, 12, 36...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 12}, {'year'...",2024-10-07T13:20:01.128837,2016-06-24
4,https://openalex.org/W2021071494,https://doi.org/10.1016/j.compedu.2005.11.017,Evaluating Bayesian networks’ precision for de...,Evaluating Bayesian networks’ precision for de...,2005,2005-12-28,{'openalex': 'https://openalex.org/W2021071494...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,[],[],26,"[https://openalex.org/W139006800, https://open...","[https://openalex.org/W581389233, https://open...","{'Students': [0], 'are': [1, 130], 'characteri...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2024, 'cited_by_count': 10}, {'year'...",2024-10-23T18:18:47.723921,2016-06-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,https://openalex.org/W2795891121,https://doi.org/10.1016/j.scitotenv.2018.03.303,Efficiency of surfactant-enhanced bioremediati...,Efficiency of surfactant-enhanced bioremediati...,2018,2018-04-06,{'openalex': 'https://openalex.org/W2795891121...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,[],[],53,"[https://openalex.org/W1528466393, https://ope...","[https://openalex.org/W4295008747, https://ope...","{'Shifts': [0], 'in': [1, 19, 56, 72], 'the': ...",https://api.openalex.org/works?filter=cites:W2...,"[{'yea